In [11]:
import requests
import re
from bs4 import BeautifulSoup

path = 'http://course-query.acad.ncku.edu.tw/qry/'                                                         ##Courses classification Main Webform
r = requests.get(path)
r.encoding = 'utf8'
html_doc = r.text
soup = BeautifulSoup(html_doc, 'lxml')


def name_a_has_target(tag):
    return tag.has_attr('target') and tag.name == 'a'
LinkList = []

def FindSchool(tag):
    return tag.name == 'div' and tag['class'] == ['theader']


#find first link
KeyTag = soup.find(href = re.compile('dept_no=A2'))     ##Search 'dept_no=A2', Get Tag including first-wanted link, and name it KeyTag
LnkDict = {'link': str('http://course-query.acad.ncku.edu.tw/qry/' + str(KeyTag['href'])), 
           'School': str(KeyTag.find_previous(FindSchool).string) }
LinkList.append(LnkDict)


#find all next links
count = 0
for item in KeyTag.find_all_next(name_a_has_target):
    LnkDict = {'link': str('http://course-query.acad.ncku.edu.tw/qry/' + str(item['href'])), 
               'School': str(item.find_previous(FindSchool).string) }
    LinkList.append(LnkDict)
#     print(LnkDict)
    print('link%a success ' %(count))
    count +=1

{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=A3', 'School': '其他'}
link0 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=A4', 'School': '其他'}
link1 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=A5', 'School': '其他'}
link2 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=A6', 'School': '其他'}
link3 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=AA', 'School': '其他'}
link4 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=AH', 'School': '其他'}
link5 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=AN', 'School': '不分學院'}
link6 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=C0', 'School': '不分學院'}
link7 success 
{'link': 'http://course-query.acad.ncku.edu.tw/qry/qry001.php?dept_no=A1', 'School': '通識課程'}
link8 success 
{'link': 'http://course-query.acad.ncku.

In [12]:
links = []
for di in LinkList:
    links.append(di['link'])

In [22]:
for link in enumerate(links):
    file = open('SuccessCourses/'+ str(link[0]), 'w', encoding='utf8')
    r = requests.get(link[1])
    r.encoding = 'utf8'
    html = r.text
    file.write(html)
    file.close()

In [36]:
import pandas as pd

def WriteHtmlIntoMongo(pahtLi, collection):

    ## Convert Html document to DataFrame, and then to Dictionary List(which can be directly insert into MongoDB)
    def HtmlCoursesInssertMongo(FilePath):
        print(FilePath)
        df_courses = pd.read_html(FilePath ,encoding='utf8')[0]
        df_courses = df_courses.where(df_courses.notnull(), None)
                                                        ##if df_courses.notnull() return true, df_courses = df_courses
                                                        ##else, df_courses=None
                
        ##remove escape char in the column header
        columns = []
        for column in df_courses.columns:
            columns.append(column.replace("(","").replace("*","").replace(":","").replace(")","").replace("連結課程地圖","").replace("主負責老師","").replace(" ",""))
        df_courses.columns = columns
        
        
        ##remove duplicate column header in a lot of Rows
        if len(df_courses[df_courses['系所名稱']=='系所名稱'])>0:
            length = len(df_courses[df_courses['系所名稱']=='系所名稱'])
            for i in range(0,length):
                df_courses.drop(df_courses[df_courses['系所名稱']=='系所名稱'].index[[0]], inplace=True)
                
        if len(df_courses['時間']) > 0:
            ##Replace WeekDay in Chinese
            def ReplaceWeekDay(item):
                if item != None:
                    return item.replace("[1]","一").replace("[2]","二").replace("[3]","三").replace("[4]","四").replace("[5]","五").replace("[6]","六").replace("[7]","日")
#                     return item.replace("[1]","一").replace("[2]","二").replace("[3]","三").replace("[4]","四").replace("[5]","五").replace("[6]","六").replace("[7]","日").replace("~","_")
            df_courses['時間'] = df_courses['時間'].apply(ReplaceWeekDay)
    
        if len(df_courses['教師姓名']) > 0:
            ##Replace TeacherName with special char "*" with "#"
            def ReplaceTeacherName(item):
                if item != None:
                    return item.replace("*","(主),").replace(" ",",")
            df_courses['教師姓名'] = df_courses['教師姓名'].apply(ReplaceTeacherName)
            
        
        ##Mongo allow dictionary only. Before transfer to dict, first store it in string type, otherwise, the type in Mongo can be various
        def ToDIct(Row):
            def ToStrAndRemoveSpace(item):
                if(item != None):
                    return str(item).replace(" ","")
                else:
                    return None
            return Row.apply(ToStrAndRemoveSpace).to_dict()
        CoursesDictList = df_courses.apply(ToDIct, axis=1).tolist()
        return CoursesDictList
    
    for path in PathList:
        collection.insert_many(HtmlCoursesInssertMongo(path))


import os
def ConvertToAbsolutePath(filename):
    return 'SuccessCourses/'+filename
PathList = pd.Series(os.listdir('SuccessCourses')).apply(ConvertToAbsolutePath).tolist()

from pymongo import MongoClient
conn = MongoClient('localhost', 27017)
db = conn.SuperUniversityCourses
collection = db['success']

WriteHtmlIntoMongo(PathList, collection)   ##Use function defined above

conn.close()

SuccessCourses/0
SuccessCourses/1
SuccessCourses/10
SuccessCourses/100
SuccessCourses/101
SuccessCourses/102
SuccessCourses/103
SuccessCourses/104
SuccessCourses/105
SuccessCourses/106
SuccessCourses/107
SuccessCourses/108
SuccessCourses/109
SuccessCourses/11
SuccessCourses/110
SuccessCourses/111
SuccessCourses/112
SuccessCourses/113
SuccessCourses/114
SuccessCourses/115
SuccessCourses/116
SuccessCourses/117
SuccessCourses/118
SuccessCourses/119
SuccessCourses/12
SuccessCourses/120
SuccessCourses/121
SuccessCourses/122
SuccessCourses/123
SuccessCourses/124
SuccessCourses/125
SuccessCourses/126
SuccessCourses/127
SuccessCourses/128
SuccessCourses/129
SuccessCourses/13
SuccessCourses/130
SuccessCourses/131
SuccessCourses/132
SuccessCourses/133
SuccessCourses/134
SuccessCourses/135
SuccessCourses/136
SuccessCourses/137
SuccessCourses/138
SuccessCourses/139
SuccessCourses/14
SuccessCourses/140
SuccessCourses/141
SuccessCourses/142
SuccessCourses/143
SuccessCourses/144
SuccessCourses/145
Su

In [31]:
import pandas as pd

df_courses = pd.read_html('SuccessCourses/30' ,encoding='utf8')[0]
df_courses = df_courses.where(df_courses.notnull(), None)


In [42]:
# df_courses.where(df_courses['系所名稱']=='系所名稱')

In [68]:
##append new Row

df_courses = df_courses.append(df_courses[df_courses['系所名稱']=='系所名稱'])
# df_courses.append(df_courses[df_courses['系所名稱']=='系所名稱'].to_dict())

# df_courses.drop(df_courses[df_courses['系所名稱']=='系所名稱'].index[[0]])

In [92]:
# df_courses[df_courses['系所名稱']=='系所名稱'].index[[0]]
for i in range(0,len(df_courses[df_courses['系所名稱']=='系所名稱'])):
    df_courses.drop(df_courses[df_courses['系所名稱']=='系所名稱'].index[[0]], inplace=True)    

In [93]:
# df_courses.describe()
# len(df_courses[df_courses['系所名稱']=='系所名稱'])
df_courses[df_courses['系所名稱']=='系所名稱']

Empty DataFrame
Columns: [系所名稱, 系號, 序號, 課程碼, 分班碼, 班別, 年級, 類別, 組別, 英語授課, 課程名稱(連結課程地圖), 選必修, 學分, 教師姓名*:主負責老師, 已選課人數 , 餘額 , 時間, 教室, 備註, 限選條件, 業界專家參與, 屬性碼, 跨領域學分學程, Moocs]
Index: []

[0 rows x 24 columns]

In [102]:
columns = []
for column in df_courses.columns:
    columns.append(column.replace("(","").replace("*","").replace(":","").replace(")","").replace("連結課程地圖","").replace("主負責老師","").replace(" ",""))

In [106]:
df_courses.columns = columns

In [6]:

def aaa(item):
    return str(item)
df_courses = df_courses.apply(aaa)

系所名稱            0     化學所 CHEM\n1     化學所 CHEM\n2     化學所 CHEM...
系號              0     L3\n1     L3\n2     L3\n3     L3\n4     ...
序號              0     001\n1     002\n2     005\n3     013\n4 ...
課程碼             0     L350420\n1     L350420\n2     L360100\n3...
分班碼             0        1\n1        2\n2        1\n3     None...
班別              0     碩一\n1     碩一\n2      碩\n3     碩博\n4     ...
年級              0     None\n1     None\n2     None\n3     None...
類別              0     講義\n1     講義\n2     講義\n3     講義\n4     ...
組別              0     None\n1     None\n2     None\n3     None...
英語授課            0        N\n1        N\n2        N\n3        N...
課程名稱(連結課程地圖)    0          書報討論（二）\n1          書報討論（二）\n2     ...
選必修             0      必修\n1      必修\n2      必修\n3      選修\n4 ...
學分              0      1\n1      1\n2      0\n3      3\n4     ...
教師姓名*:主負責老師     0                         吳欣倫\n1              ...
已選課人數           0        23\n1        24\n2       110\n3      ...
餘額        

In [27]:
def tostr(item):
    return str(item).replace(" ","")
df_courses.xs(0).apply(tostr)

系所名稱              化學所CHEM
系號                     L3
序號                    001
課程碼               L350420
分班碼                     1
班別                     碩一
年級                   None
類別                     講義
組別                   None
英語授課                    N
課程名稱(連結課程地圖)      書報討論（二）
選必修                    必修
學分                      1
教師姓名*:主負責老師           吳欣倫
已選課人數                  23
餘額                     33
時間                   [2]5
教室              理化大樓35306
備註                   None
限選條件                 None
業界專家參與                  否
屬性碼              CHEM7095
跨領域學分學程              None
Moocs                   否
Name: 0, dtype: object

In [34]:
type(df_courses)

pandas.core.frame.DataFrame